<a href="https://colab.research.google.com/github/brendanpshea/database_sql/blob/main/Database_09_PokemonAndPostgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Insteall postgres
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%sql postgresql+psycopg2://@/postgres

 * Starting PostgreSQL 14 database server
   ...done.
ERROR:  role "root" already exists


## Overview of the Pokemon Clinic Database:

To demonstrate the power of Postgres, we'll be creating a The Pokemon Clinic database consists of three main tables: `trainers`, `pokemon`, and `medical_records`. These tables are designed to store information about Pokemon trainers, their Pokemon, and the medical records associated with each Pokemon.

In [7]:
%%sql
-- Create the trainers table
DROP TABLE IF EXISTS trainers CASCADE;
DROP TABLE IF EXISTS pokemon CASCADE;
DROP TABLE IF EXISTS medical_records;

-- Create the trainers table
CREATE TABLE trainers (
  id SERIAL PRIMARY KEY,
  name VARCHAR(100) NOT NULL,
  email VARCHAR(100) UNIQUE NOT NULL,
  password_hash VARCHAR(100) NOT NULL,
  phone VARCHAR(20),
  date_of_birth DATE,
  CHECK (date_of_birth < CURRENT_DATE)
);

-- Create the pokemon table
CREATE TABLE pokemon (
  id SERIAL PRIMARY KEY,
  name VARCHAR(50) NOT NULL,
  species VARCHAR(50) NOT NULL,
  trainer_id INTEGER REFERENCES trainers(id),
  level INTEGER CHECK (level BETWEEN 1 AND 100),
  health_status VARCHAR(20),
  abilities TEXT[]
);

-- Create the medical_records table
CREATE TABLE medical_records (
  id SERIAL PRIMARY KEY,
  pokemon_id INTEGER REFERENCES pokemon(id),
  visit_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  diagnosis TEXT,
  treatment TEXT,
  cost NUMERIC(8, 2)
);

 * postgresql+psycopg2://@/postgres
Done.
Done.
Done.
Done.
Done.
Done.


[]

While much of this statement should be familiar, you might notice a few new things, as well.

The `trainers` table stores information about Pokemon trainers.
  -   It includes columns for the trainer's ID (auto-generated), name, email (unique), hashed password, phone number, and date of birth.
  -   The `date_of_birth` column has a CHECK constraint that ensures the value is always less than the current date. This constraint uses the `CURRENT_DATE` function, which is specific to PostgreSQL and not available in SQLite.
  -   The `email` column has a UNIQUE constraint to ensure that each email address is associated with only one trainer.

The pokemon` table stores information about individual Pokemon.
  -   It includes columns for the Pokemon's ID (auto-generated), name, species, trainer ID (foreign key referencing the `trainers` table), level, health status, and abilities.
   -   The `level` column has a CHECK constraint that ensures the value is between 1 and 100.
  -   The `abilities` column is of type TEXT[] (array), which allows storing multiple abilities for each Pokemon. Arrays are a feature specific to PostgreSQL and not available in SQLite.

The `medical_records` table stores information about medical visits and treatments for each Pokemon.
  -   It includes columns for the medical record ID (auto-generated), Pokemon ID (foreign key referencing the `pokemon` table), visit date, diagnosis, treatment, and cost.
  -   The `visit_date` column has a DEFAULT value of `CURRENT_TIMESTAMP`, which automatically sets the value to the current timestamp if no value is provided during insertion. This is a PostgreSQL-specific feature.
  -   The `cost` column is of type NUMERIC(8, 2), which allows storing monetary values with a precision of 8 digits and 2 decimal places. The NUMERIC type provides more precise decimal calculations compared to SQLite's REAL type.

### Test Data for Pokemon Clinic
Now, let's insert some test data for our Pokemon Clinic

In [9]:
%%sql
CREATE EXTENSION IF NOT EXISTS pgcrypto;
-- Insert sample data into the trainers table
INSERT INTO trainers (name, email, password_hash, phone, date_of_birth)
VALUES
  ('Ash Ketchum', 'ash@example.com', crypt('password1', gen_salt('bf')), '123-456-7890', '1990-05-22'),
  ('Misty', 'misty@example.com', crypt('password2', gen_salt('bf')), '987-654-3210', '1991-07-19');

-- Insert sample data into the pokemon table
INSERT INTO pokemon (name, species, trainer_id, level, health_status, abilities)
VALUES
  ('Pikachu', 'Pikachu', 1, 25, 'Healthy', ARRAY['Static', 'Lightning Rod']),
  ('Charizard', 'Charizard', 1, 65, 'Healthy', ARRAY['Blaze']),
  ('Staryu', 'Staryu', 2, 30, 'Injured', ARRAY['Natural Cure', 'Illuminate']);

-- Insert sample data into the medical_records table
INSERT INTO medical_records (pokemon_id, diagnosis, treatment, cost)
VALUES
  (3, 'Minor abrasions', 'Applied healing salve', 50.00),
  (1, 'Regular check-up', 'None', 0.00);

 * postgresql+psycopg2://@/postgres
Done.
2 rows affected.
3 rows affected.
2 rows affected.


[]

Again, much of this shoudl be familar. However, if you look closely at the ways the trainer's passwords are handled, you'll notice something a bit different.

- The `password_hash` column stores the hashed version of the trainer's password. **Hashing** is a one-way process that converts the plain-text password into a fixed-size string of characters. The resulting hash is irreversible, meaning it is computationally infeasible to obtain the original password from the hash.
- In this example, the `crypt` function is used along with the `gen_salt` function to hash the passwords. The `crypt` function applies a cryptographic hash function to the password, and the `gen_salt` function generates a random salt value using the Blowfish ('bf') algorithm. The salt is appended to the password before hashing, making it more resistant to rainbow table attacks and increasing the security of stored passwords.
- Storing hashed passwords instead of plain-text passwords is crucial for security. If the database is compromised, attackers would only have access to the hashed passwords, making it extremely difficult for them to retrieve the original passwords.

It's important to note that while the example uses the `crypt` function for simplicity, in a production environment, it's recommended to use more secure and modern hashing algorithms specifically designed for password hashing, such as bcrypt, scrypt, or PBKDF2, which provide better protection against various types of attacks.

We see what this hash looks like:

In [10]:
%%sql
SELECT * FROM trainers;

 * postgresql+psycopg2://@/postgres
2 rows affected.


id,name,email,password_hash,phone,date_of_birth
1,Ash Ketchum,ash@example.com,$2a$06$8D1p1/.vbeVQVKI4nsthWOa5x/kUmgb59TNAC.lZ91NpCzVTnvtHa,123-456-7890,1990-05-22
2,Misty,misty@example.com,$2a$06$eLSwPwVKwJYx3VIGwHbpvOAcE3R/YjBVUx.EPwAHo9EgiHVu0wqpe,987-654-3210,1991-07-19
